In [1]:
from pathlib import Path

def fetch_subsequent_dir(data_root: Path, target_subdir_name: Path):
    data_paths = list(data_root.glob(f"*/{target_subdir_name}"))
    # display(data_paths)
    # get sub dir - no target_subdir_name
    subdir_path_list = [data_path.parent for data_path in data_paths]
    # display(subdir_path_list)
    subdir_name_list = [subdir.name for subdir in subdir_path_list]
    # display(subdir_name_list)
    return subdir_path_list, subdir_name_list


# data_root = Path("/home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/00_test")
# data_root = Path("/home/tianqi/zhoulab/Insect_Trap_Project/Result_16mp")
data_root = Path("/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7")

target_subdir_name = Path("raw_data")
subdir_path_list, subdir_name_list = fetch_subsequent_dir(data_root, target_subdir_name)
display(subdir_path_list)
display(subdir_name_list)
len(subdir_path_list), len(subdir_name_list)

[PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0613-0624_04_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0624-0702_01_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0730-0813_01_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/lloyd_0715-0729_04_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0605-0621_40_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0710-0726_36_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0726-0809_11_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0809-0823_34_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms2_0621-0710_01_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms2_0726-0809_13_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/ru

['jeff_0613-0624_04_ok',
 'jeff_0624-0702_01_ok',
 'jeff_0730-0813_01_ok',
 'lloyd_0715-0729_04_ok',
 'ms1_0605-0621_40_ok',
 'ms1_0710-0726_36_ok',
 'ms1_0726-0809_11_ok',
 'ms1_0809-0823_34_ok',
 'ms2_0621-0710_01_ok',
 'ms2_0726-0809_13_ok',
 'ms2_0809-0823_10_ok',
 'sw1_0605-0613_07_ok',
 'sw1_0711-0725_03_ok',
 'sw2_0725-0808_02_ok',
 'sw2_0808-0823_04_ok']

(15, 15)

In [2]:
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go

def draw(csv_file_root_path: Path, connectgaps, show_plot: bool = True):
    """
    从指定根目录下的 12_statistics_with_full_time.csv 读取数据，
    利用已有的 sample_id 列构造 FiftyOne URL，并绘制交互式时间序列图。

    参数：
        csv_file_root_path: 例如 Path("/.../ms1_0726-0809_11_ok")
        connectgaps: 传给 plotly Scatter 的 connectgaps 参数（True/False）
        show_plot: 是否在 Notebook 中直接显示图像
    """
    # 1) 定位 CSV 路径
    csv_file_path = csv_file_root_path / "raw_data_sliced_merge/pose_and_det_v1/12_statistics_with_full_time.csv"
    if not csv_file_path.exists():
        print(f"文件不存在，跳过: {csv_file_path}")
        return

    csv_parent_path = csv_file_path.parent

    # 2) 读 CSV（假设已经有 sample_id 列）
    df = pd.read_csv(csv_file_path, parse_dates=["time"])
    df = df.sort_values("time")

    # 如果没有 sample_id 列，可以给个提示（防止忘记）
    if "sample_id" not in df.columns:
        print("⚠ CSV 中没有 sample_id 列，无法构造 FiftyOne URL，只画图不跳转")
        df["sample_id"] = None

    # 3) 根据 sample_id 拼 URL
    dataset_name = csv_file_root_path.name
    base_url = f"https://fiftyone.tianqiyao.men/datasets/{dataset_name}"

    def make_url(sid):
        if pd.isna(sid) or sid is None:
            return None
        # 如果 sample_id 是数字，转成字符串
        sid_str = str(sid)
        return f"{base_url}?id={sid_str}"

    df["url"] = df["sample_id"].map(make_url)

    # 4) 画图
    fig = go.Figure()

    # 间隔计数散点（可点击打开对应 FiftyOne 样本）
    fig.add_trace(
        go.Scatter(
            x=df["time"],
            y=df["swd_count"],
            mode="markers",
            name="SWD count (interval)",
            line=dict(width=1.8, color="#FF7F0E"),
            marker=dict(size=4),
            yaxis="y1",
            customdata=df[["url","original_name","time"]].values,  # 点击时用这个 URL
            hovertemplate=(
                "<b>swd count</b>: %{y}<br>"
                "Filename: %{customdata[1]}<br>"
                "Time: %{customdata[2]}<br>"
                "Click This Point To Open The Link<br>"
                # "<extra></extra>"
            ),
        )
    )

    # 累积计数曲线
    fig.add_trace(
        go.Scatter(
            x=df["time"],
            y=df["swd_cumu"],
            mode="lines",
            name="SWD cumulative",
            line=dict(width=2.2, color="#1F77B4"),
            yaxis="y2",
            connectgaps=connectgaps,
        )
    )

    xaxis_dict = dict(
        title="Time",
        rangeslider=dict(visible=True),
    )

    fig.update_layout(
        xaxis=xaxis_dict,
        yaxis=dict(
            title="SWD count / interval",
            side="left",
            range=[0, max(df["swd_count"].fillna(0)) * 1.2],
        ),
        yaxis2=dict(
            title="Cumulative count",
            overlaying="y",
            side="right",
            showgrid=False,
            range=[0, max(df["swd_cumu"].fillna(0)) * 1.2],
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
        ),
        template="plotly_white",
        margin=dict(l=70, r=70, t=60, b=40),
        title="SWD Time Series",
    )

    # Notebook 中显示
    if show_plot:
        fig.show()

    # 5) 导出 HTML
    output_html = csv_parent_path / "swd_time_series_plot.html"

    # 这个 JS 会在点击任意点时打开 customdata 中的 URL
    post_script = """
    document.addEventListener("DOMContentLoaded", function() {
        var plot = document.getElementsByClassName('js-plotly-plot')[0];
        if (!plot) { return; }

        plot.on('plotly_click', function(data) {
            if (!data || !data.points || !data.points.length) { return; }
            var point = data.points[0];
            var url = point.customdata[0];  // customdata 中的第一个元素是 URL
            if (url) {
                window.open(url, '_blank');
            }
        });
    });
    """

    fig.write_html(
        output_html,
        include_plotlyjs='cdn',  # 使用 CDN 以减小文件大小
        full_html=True,
        post_script=post_script,  # 注入点击事件
    )
    print(f"HTML 已导出到: {output_html}")


In [3]:
# ==== 1. 读取数据 ====
# 把文件名改成你自己的 CSV 路径
# csv_file_path = "/home/tianqi/zhoulab/Insect_Trap_Project/Result_16mp/ms1_0726-0809_11_ok/processed_data.csv"
csv_file_root_path = Path("/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0726-0809_11_ok")


draw(csv_file_root_path, connectgaps=True)

/tmp/ipykernel_4117657/4128750164.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(csv_file_path, parse_dates=["time"])


HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0726-0809_11_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html


In [4]:
for subdir_path in subdir_path_list:
    # ==== 1. 读取数据 ====
    draw(subdir_path, connectgaps=True, show_plot=False)


HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0613-0624_04_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html
⚠ CSV 中没有 sample_id 列，无法构造 FiftyOne URL，只画图不跳转
HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0624-0702_01_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html
文件不存在，跳过: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0730-0813_01_ok/raw_data_sliced_merge/pose_and_det_v1/12_statistics_with_full_time.csv


/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/lloyd_0715-0729_04_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html
HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0605-0621_40_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html


/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0710-0726_36_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html
HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0726-0809_11_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html


...

HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms2_0726-0809_13_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html
HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms2_0809-0823_10_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html
HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/sw1_0605-0613_07_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html


/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/sw1_0711-0725_03_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html
⚠ CSV 中没有 sample_id 列，无法构造 FiftyOne URL，只画图不跳转
HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/sw2_0725-0808_02_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html


/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipykernel_4117657/4128750164.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



HTML 已导出到: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/sw2_0808-0823_04_ok/raw_data_sliced_merge/pose_and_det_v1/swd_time_series_plot.html
